In [21]:
#Make sure your are inside the folder
#-L: level of details
#-C: color for folders
!tree -L 2 -C

.
├── README
├── Tutorial-Copy1.ipynb
├── Tutorial.ipynb
├── Untitled.ipynb
├── imdb.vocab
├── imdbEr.txt
├── test
│   ├── labeledBow.feat
│   ├── neg
│   ├── pos
│   ├── urls_neg.txt
│   └── urls_pos.txt
└── train
    ├── labeledBow.feat
    ├── neg
    ├── pos
    ├── unsup
    ├── unsupBow.feat
    ├── urls_neg.txt
    ├── urls_pos.txt
    └── urls_unsup.txt

7 directories, 14 files


In [22]:
from sklearn.datasets import load_files
review_train = load_files('train/',categories=['neg','pos'])
type(review_train)

sklearn.utils.Bunch

In [23]:
review_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [33]:
X,y = review_train.data, review_train.target
print(len(X), y.shape)

25000 (25000,)


In [34]:
print(X[0])

b"Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty."


In [35]:
#Remove newline break <br />
X = [txt.replace(b'<br />',b'') for txt in X]
print(X[0])

b"Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. Flawed but honest with a terrible honesty."


In [36]:
#checking y: first 10 and last 10 elements of y
print(y[:10],y[-10:])

[1 0 1 0 0 1 1 0 0 1] [1 1 1 0 1 1 1 0 0 0]


In [64]:
#NLP, text processing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = CountVectorizer()
#vect = TfidfVectorizer(ngram_range=(1,2), min_df=3)

vect.fit(X)
x = vect.transform(X)
type(x)
#print('Text processing complete!')

scipy.sparse.csr.csr_matrix

In [65]:
#How many words in my vocabulary
#What are they?

vocabulary = vect.get_feature_names()
print(type(vocabulary))
print(len(vocabulary))
print(vocabulary[:10])
print(vocabulary[-10:])

#The most common / least common words
import numpy as np
words_total_counts = np.sum(x, axis=0)
print(type(words_total_counts))
print(words_total_counts)

#np.sort vs np.argsort()
idx_sorted_by_counts = np.argsort(words_total_counts)
print(idx_sorted_by_counts)

#list comprehension
most_common_words  = [vocabulary[idx_sorted_by_counts[0,i]] for i in range(-10,0)]
most_common_counts = [words_total_counts[0,idx_sorted_by_counts[0,i]] for i in range(-10,0)]

least_common_words  = [vocabulary[idx_sorted_by_counts[0,i]] for i in range(10)]
least_common_counts = [words_total_counts[0,idx_sorted_by_counts[0,i]] for i in range(10)]

print(list(zip(most_common_words,most_common_counts)))
print(list(zip(least_common_words,least_common_counts)))

<class 'list'>
75911
['00', '000', '0000000000001', '00001', '00015', '000s', '001', '003830', '006', '007']
['était', 'état', 'étc', 'évery', 'êxtase', 'ís', 'ísnt', 'østbye', 'über', 'üvegtigris']
<class 'numpy.matrixlib.defmatrix.matrix'>
[[ 93 300   1 ...,   1   4   1]]
[[37955 39352 39354 ..., 47352  3375 67244]]
[('was', 48198), ('that', 73274), ('this', 75965), ('in', 93955), ('it', 96436), ('is', 107313), ('to', 135706), ('of', 145851), ('and', 164109), ('the', 336633)]
[('labourers', 1), ('lilienthal', 1), ('lilililililii', 1), ('lilleheie', 1), ('lillete', 1), ('lilley', 1), ('lillihamer', 1), ('liman', 1), ('likings', 1), ('limbless', 1)]


In [68]:
#Use LogisticRegression to train the classifier
# x, y
from sklearn.linear_model import LogisticRegression
lgr = LogisticRegression()
lgr.fit(x,y) #training done!

#predict
review_test = load_files('test/',categories=['neg', 'pos'])
X_test, y_test = review_test.data, review_test.target
x_test = vect.transform(X_test)

In [51]:
#training score
lgr.score(x,y)

0.99831999999999999

In [69]:
#test score
from sklearn.metrics import accuracy_score
y_pred = lgr.predict(x_test)
print(accuracy_score(y_pred, y_test))

0.86704


In [70]:
#Regularization parameter C
#reduce overfitting
from sklearn.model_selection import GridSearchCV
param = {'C': [0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param, cv=5, n_jobs=-1)
grid.fit(x,y)

print(grid.best_params_)
print(grid.best_score_)

{'C': 0.1}
0.88844


In [71]:
#Improved accuracy
y_pred = grid.predict(x_test)
print(accuracy_score(y_pred, y_test))

0.87892


In [72]:
#Use a different Text processing technique: Tfidf vectorization
#vect = CountVectorizer()
vect = TfidfVectorizer(ngram_range=(1,2), min_df=3)

vect.fit(X)
x = vect.transform(X)
type(x)

x_test = vect.transform(X_test)

param = {'C': [0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param, cv=5, n_jobs=-1)
grid.fit(x,y)

y_pred = grid.predict(x_test)
print(accuracy_score(y_pred, y_test))
#Much improved accuracy with Tfidf!!! 0.907

scipy.sparse.csr.csr_matrix

Play around and have some fun!